In [ ]:
 import os
import csv
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import selenium
import re
import pandas as pd
import random
from selenium.webdriver.common.keys import Keys


### Download Chrome WebDriver:

First,we downloaded the Chrome WebDriver that corresponds to our installed version of Google Chrome. We downloaded it from the official ChromeDriver download page: https://chromedriver.chromium.org/downloads


Note :
Be sure to download the version that matches your Chrome browser version. You can check your Chrome version by clicking on the three vertical dots in the top-right corner of the browser, then going to "Help" > "About Google Chrome."

We also used a VPN and kept rotating the it so that redfin doesnot identify this as bot request

In [ ]:
#Specify the path to the downloaded Chrome WebDriver
cpath = "/Users/aasthatandon/chromedriver-mac-arm64/chromedriver"


In [ ]:
'''This function is intended to configure the Selenium WebDriver
for Google Chrome with certain settings to bypass or
minimize detection of automation or bot activity.'''

def setup_chrome():
    # Create Chromeoptions instance
    options = webdriver.ChromeOptions()

    # Adding argument to disable the AutomationControlled flag
    options.add_argument("--disable-blink-features=AutomationControlled")

    # Exclude the collection of enable-automation switches
    options.add_experimental_option("excludeSwitches", ["enable-automation"])

    # Turn-off userAutomationExtension
    options.add_experimental_option("useAutomationExtension", False)

    # Setting the driver path and requesting a page
    driver = webdriver.Chrome(cpath, options=options)

    # Changing the property of the navigator value for webdriver to undefined
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    return driver

In [ ]:
'''
Here we are using Selenium WebDriver
to automate actions on the Redfin website,
including searching for a location
interacting with specific elements and downloading
the initial data.
'''
#Set up the chrome driver
driver = setup_chrome()

# Open the Redfin website in the Chrome browser
driver.get('https://www.redfin.com/')

# Maximize the Chrome browser window
driver.maximize_window()

# Find and interact with the search input field on the Redfin website using its XPath.
element = driver.find_element("xpath","//input[@title='City, Address, School, Agent, ZIP']")

# Click and clear on the search input field
element.click()
element.clear()

# Input the search query, in this case, "san jose," into the search field.
element.send_keys("san jose")

# Simulate 'RETURN' key to initiate the search.
element.send_keys(Keys.RETURN)

# Pause the execution for 7.5 seconds to allow the webpage to load
time.sleep(7.5)

# Find and interact with the 'download-and-save' button using its XPath.
element = driver.find_element("xpath","//a[@id='download-and-save']")

# Scroll to the 'download-and-save' button in the webpage.
driver.execute_script("arguments[0].scrollIntoView({behavior: 'auto', block: 'center', inline: 'center'});", element)

# Click on the 'download-and-save' button.
element.click()

# Pause the script execution for 6.2 seconds, for download to complete.
time.sleep(6.2)

# Close the Chrome browser.
driver.close()


In this initial step, Step 1, data is downloaded to the "download" folder, and each file is named in the "redfin_date_time" format .Following this, we organize the data into specific folders for future use, where we can easily import it into a dataframe. This sets the stage for the Step 2, where we would extract additional information about each house from this Step1 data by hitting their URLs.

In a similar way, we  gathered Step 1 data for different locations by replacing the search query in element.send_keys(). We conducted searches for multiple areas, including but not limited to: Fremont, Sunnyvale, San Jose, Mountain View, Palo Alto, Hayward, Union City, Newark, Menlo Park, Los Altos, Los Gatos, Morgan Hill, Gilroy, Milpitas, Campbell, San Francisco, Oakland, Concord, Cupertino, Dublin, Foster City, Livermore, Dublin, Pleasanton, Redwood City, Santa Clara, Irvine,  and more.

In [ ]:
# Reading the step 1 data into a dataframe
df_step1 = pd.read_csv('/Users/aasthatandon/Library/Mobile Documents/com~apple~CloudDocs/CSUEB materials/Fall 2023/BAN 612/Redfin/Version2/Data/California/BayArea/San Jose/Step1/redfin_2023-09-29-20-15-24.csv')

In [ ]:
# Renaming the URL column
df_step1 = df_step1.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)' : 'URL'})

In [ ]:
# Looking into the columns extracted in Step 1
df_step1.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [ ]:
df_step1.shape # Looking into the shape of dataframe

(350, 27)

In [ ]:
'''
We limited the number of houses to 100
for locations with more than 100 listings.
Conversely, if a location had fewer than 100
houses listed, we included all of them.
The count of houses per location would be determined
in the data preprocessing phase.
'''
df_step1 = df_step1[:100]
df_step1.shape

(100, 27)

In [ ]:
#Looking into the initial rows of step1 dataframe
df_step1.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,MLS Listing,NaN,Single Family Residential,6544 Little Falls Dr,San Jose,CA,95120,2378999,4.0,2.5,...,Active,September-30-2023 02:00 PM,September-30-2023 04:00 PM,https://www.redfin.com/CA/San-Jose/6544-Little...,MLSListings,ML81940360,N,Y,37.216909,-121.870003
1,MLS Listing,NaN,Condo/Co-op,922 Gaspar Vis,San Jose,CA,95126,1200000,4.0,4.0,...,Active,September-30-2023 01:00 PM,September-30-2023 04:00 PM,https://www.redfin.com/CA/San-Jose/922-Gaspar-...,MLSListings,ML81940163,N,Y,37.321338,-121.904130
2,MLS Listing,NaN,Single Family Residential,4993 Grimsby Dr,San Jose,CA,95130,2498888,4.0,2.5,...,Active,September-30-2023 11:00 AM,September-30-2023 02:00 PM,https://www.redfin.com/CA/San-Jose/4993-Grimsb...,MLSListings,ML81943157,N,Y,37.280443,-121.988676
3,MLS Listing,NaN,Single Family Residential,818 N 13th St,San Jose,CA,95112,1118000,3.0,1.0,...,Active,NaN,NaN,https://www.redfin.com/CA/San-Jose/818-N-13th-...,MLSListings,ML81943043,N,Y,37.357811,-121.889502
4,MLS Listing,NaN,Single Family Residential,2451 Cory Ave,San Jose,CA,95128,899999,3.0,1.0,...,Active,NaN,NaN,https://www.redfin.com/CA/San-Jose/2451-Cory-A...,MLSListings,ML81942790,N,Y,37.331982,-121.948778


In [ ]:

'''Function to extract house informations like
Parking Description,Utilities,Dining Room Description,
Garage,Heating Feature,Cooling Feature,Flooring feature,
Laundary Feature,Number of stories,Fees,
Bathroom description,HOA Freq,HOA Name
'''

def get_house_info(soup):
    """
    soup: Beautiful.soup object
    """
    house_info = {}

    card = "entryItemContent"
    tag = soup.findAll('span', attrs={'class': card})
    for idx,val in enumerate(tag):
        tag_text = val.get_text()


        #Parking Description
        park_regex = r'Parking Description: (.+)'
        pattern_park = re.compile(park_regex)
        park = re.findall(pattern_park,tag_text)
        if park:
            house_info['PARK_DESCRIPTION']=park[0]

        #Utilities
        utl_regex = r'Utilities: (.*)'
        pattern_utl = re.compile(utl_regex)
        utl = re.findall(pattern_utl,tag_text)
        if utl:
            house_info['UTILITIES']=utl[0]

        # Dining Room Description
        din_regex = r'Dining Room Description: (.*)'
        pattern_din = re.compile(din_regex)
        din = re.findall(pattern_din,tag_text)
        if din:
            house_info['DINNING_DESC']=din[0]


        # Garage
        garage_regex = r'Garage Spaces: (\d+)'
        garage_pattern = re.compile(garage_regex)
        has_garage = re.findall(garage_pattern,tag_text)
        if has_garage:
            house_info['HAS_GARAGE'] = float(has_garage[0])

        # Heating Feature
        heating_regex = r'Heating: (.*)'
        heating_pattern = re.compile(heating_regex)
        heating_features = re.findall(heating_pattern,tag_text)
        if heating_features:
            house_info['HEATING_FEATURES'] = heating_features[0]

        # Cooling Feature
        cooling_regex = r'Cooling: (.*)'
        cooling_pattern = re.compile(cooling_regex)
        cooling_features = re.findall(cooling_pattern,tag_text)
        if cooling_features:
            house_info['COOLING_FEATURES'] = cooling_features[0]

        #Flooring feature
        floor_regex = r'Flooring: (.*)'
        floor_pattern = re.compile(floor_regex)
        floor_features = re.findall(floor_pattern,tag_text)
        if floor_features:
            house_info['FLOORING_FEATURES'] = floor_features[0]

        #Laundary Feature
        laundary_regex = r'Laundry: (.*)'
        laundary_pattern = re.compile(laundary_regex)
        laundary_features = re.findall(laundary_pattern,tag_text)
        if laundary_features:
            house_info['LAUNDARY_FEATURES'] = laundary_features[0]

        # Number of stories
        stories_regex = r'# of Stories: (.*)'
        stories_pattern = re.compile(stories_regex)
        num_stories = re.findall(stories_pattern,tag_text)
        if num_stories:
            house_info['NUM_STORIES']= float(num_stories[0])

        #Fees
        fee_regex = r'Fees Include: (.*)'
        fee_pattern = re.compile(fee_regex)
        fee_features = re.findall(fee_pattern,tag_text)
        if fee_features:
            house_info['FEES_INCLUDE'] = fee_features[0]


        #Bathroom description
        bath_regex = r'Bathroom Description: (.*)'
        bath_pattern = re.compile(bath_regex)
        bath_features = re.findall(bath_pattern,tag_text)
        if bath_features:
            house_info['BATH_DESCRIPTION'] = bath_features[0]

        # HOA Freq
        hoa_regex = r'HOA Fee Frequency: (.*)'
        hoa_pattern = re.compile(hoa_regex)
        hoa_fees_freq = re.findall(hoa_pattern,tag_text)
        if hoa_fees_freq:
            house_info['HOA_FEE_FREQ'] = hoa_fees_freq[0]

        # HOA Name
        hoa_name_regex = r'HOA Name: (.*)'
        hoa_name_pattern = re.compile(hoa_name_regex)
        hoa_name = re.findall(hoa_name_pattern,tag_text)
        if hoa_name:
            house_info['HOA_NAME'] = hoa_name[0]

    return house_info

In [ ]:

'''Function to extract price information like
Buyer agency compensation,price/sqft,Redfin Estimate,
Estimated monthly pay
'''

def get_price_info(soup):
    """
    soup: Beautiful.soup object
    """
    price_info = {}

    card = "keyDetailsList"
    tag = soup.findAll('div', attrs={'class': card})

    for idx,val in enumerate(tag):
        tag_text = val.get_text()


        # Buyer agency compensation
        regex_buy = r'Buyer’s Agent Commission(\d{1,2}(?:\.\d{1,2})?)%'
        pattern_buy = re.compile(regex_buy)
        buy = re.findall(pattern_buy,tag_text)
        if buy:
            price_info['BUYER_AGENCY_COMPENSATION']=float(buy[0])

        #price/sqft
        regex_price = r'Price/Sq\.Ft\.\$(\d{1,3}(?:,\d{3})*)'
        pattern_price = re.compile(regex_price)
        price = re.findall(pattern_price,tag_text)
        if price:
            price_info['PRICE_PR_SQFT']=float(price[0].replace(',',''))

        # Redfin Estimate
        red_est_regex = r'Redfin Estimate\$(\d{1,3}(?:,\d{3})*(?:,\d{3})*)'
        red_est_pattern = re.compile(red_est_regex)
        red_estimate = re.findall(red_est_pattern,tag_text)
        if red_estimate:
            estimate = float(red_estimate[0].replace(',',''))
            price_info['REDFIN_ESTIMATE']= estimate

        # Estimated monthly pay
        month_pay = r'Est\. Mo\. Payment\$(\d{1,3}(?:,\d{3})*)'
        est_pay_pattern = re.compile(month_pay)
        estpay_month = re.findall(est_pay_pattern,tag_text)
        if estpay_month:
            pay = float(estpay_month[0].replace(',',''))
            price_info['EST_MONTHLY_PAY'] = pay




    return price_info

In [ ]:
''' Function to extract school information, mainly
school rating based on the grades.
'''

def extract_school_info(soup):
    card = "schools-content"
    tag = soup.findAll('div', attrs={'class': card})
    school_info = {'ELEMENTARY_SCHOOL_RATING': 'NA',
                  'MIDDLE_SCHOOL_RATING': 'NA',
                  'HIGH_SCHOOL_RATING': 'NA'}
    if tag:
        tag_text = tag[0].get_text()
        a = tag_text.split('•')
        rating_pattern = r'(\d+)/10'



        school_info = {'ELEMENTARY_SCHOOL_RATING': 'NA',
                      'MIDDLE_SCHOOL_RATING': 'NA',
                      'HIGH_SCHOOL_RATING': 'NA'}
        for idx, val in enumerate(a):
            # Check if "K-6" or K-5 is found in the string
            if "K-6" in a[idx] or "K-5" in a[idx]:
                match = re.search(rating_pattern, a[idx])
                rating = match.group(1) if match else "NA"
                school_info['ELEMENTARY_SCHOOL_RATING'] =  rating

            elif "7-8" in a[idx] or "6-8" in a[idx]:
                match = re.search(rating_pattern, a[idx])
                rating = match.group(1) if match else "NA"
                school_info['MIDDLE_SCHOOL_RATING'] =  rating

            # Check if "9-12" is found in the string
            elif "9-12" in a[idx]:
                match = re.search(rating_pattern, a[idx])
                rating = match.group(1) if match else "NA"
                school_info['HIGH_SCHOOL_RATING'] =  rating


    return school_info



In [ ]:
'''
Function is designed to retrieve the sales history
of a property, focusing on the last recorded events.
In the event that a property has been listed on Redfin
multiple times, the function will extract specific details
for each of these listings, including the event date,
event type price.
These details will be labeled as
follows: PRICE_HISTORY1_PRICE,PRICE_HISTORY1_EVENT_DATE',
'PRICE_HISTORY1_EVENT_TYPE and PRICE_HISTORY2_PRICE etc.
HISTORY1 to be the most recent, then HISTORY2 and so on.
However, if a property has only been listed once,
the function will exclusively extract details for HISTORY1.
'''
def extract_sales_history(soup):
    card = "row PropertyHistoryEventRow"
    tag = soup.findAll('div', attrs={'class': card})
    date_regex = r'^(.*?)Date'
    price_regex = r'\$(.*?)\sPrice'
    event_regex = r"Date(.*?)(\$|—)"
    price_history = {}

    for idx,val in enumerate(tag):
        price_history
        text = tag[idx].get_text()
        date_pattern = re.compile(date_regex)
        price_pattern = re.compile(price_regex)
        event_type = re.search(event_regex,text)
        event_date = re.findall(date_pattern, text)
        price = re.findall(price_pattern, text)

        if event_date:
            price_history[f"PRICE_HISTORY{idx+1}_EVENT_DATE"] = event_date[0]
            p = 'NA'
            if event_type:
                event_type = event_type.group(1)
                price_history[f"PRICE_HISTORY{idx+1}_EVENT_TYPE"] = event_type
            if price:
                p = float(price[0].replace(',',''))
            price_history[f"PRICE_HISTORY{idx+1}_PRICE"] = p

    return price_history


In [ ]:
'''
Function to extract Bike score,Transit score
and Walk Score
'''
def extract_score(soup,score_type = "Bike Score"):
    card = "score inline-block"
    tag = soup.findAll('div', attrs={'class': card})
    score_pattern = r'(.+?)/100'
    score = {}
    for idx, val in enumerate(tag):
        if score_type in val.get_text():
            match = re.search(score_pattern, val.get_text())
            if match:
                rating = match.group(1)
                score[score_type] = rating
                score_type_pattern = rf'100(.*?)\s*{re.escape(score_type)}'
                score_type_match = re.search(score_type_pattern, tag[idx].get_text())
                if score_type_match:
                    extracted_text = score_type_match.group(1)
                    score[f'{score_type}_type'] = extracted_text
    return score


In [ ]:
'''Function to extract the level of risk factors
associated with the house like Flood Factor,Fire Factor,
Heat Factor, Wind Factor
'''
def extract_risk_factors(soup):
    card = "riskFactorItems"
    tag = soup.findAll('div', attrs={'class': card})
    text = tag[0].findAll('h3')
    risk = {}  # Initialize an empty dictionary
    for idx, val in enumerate(text):
        text = val.get_text()
        a = text.split(' - ')
        if len(a)==2:
            factor_name, factor_level = a
            risk[factor_name] = factor_level
        else:
            factor_name = a[0]
            risk[factor_name] = 'NA'

    return risk

In [ ]:
'''Function to extract the count of nearby grocery, restaurant,
transit and parks.Later on will use regex to separate them
in Data preprocessing
'''
def nearby_grocery_park_transit(soup):
    nearby_places ={}
    card="AroundThisHomeListItems row AroundThisHomeListItems__desktop"
    tag = soup.findAll('div', attrs={'class': card})
    if tag:
        text = tag[0].get_text()
        nearby_places['OVERALL_NEARBY_PLACES'] = text
    return nearby_places


### **Caution: Execute all_details={} cell only once.**
Running this again will loose all information stored and initialize an empty dictionary again

In [ ]:
all_details = {}

Next cell initializes the driver and calls all the necessary functions to extract property information from Redfin. There's a possibility that Redfin may detect this as a bot request, so we have implemented a mechanism to store all the results in the "all_details" dictionary. We also check whether a URL has already been processed to avoid redundant requests. This approach helps conserve time and effort that has already been invested.

In [ ]:
for idx, row in df_step1.iterrows():
    url = row['URL']
    if url in all_details:
        continue

    property_info = {}
    #dr = webdriver.Chrome(cpath)
    dr = setup_chrome()

    if idx % 5 == 0:
        print(f"processing {idx}")

    if idx % 5 == 0:
        # sleep time has been randomized,so as redfin
        # doesnot identify a pattern and block us
        ts = random.choice([6.9, 7.2, 8.8, 10.5, 12.3, 7.6, 10.9])
        time.sleep(ts)

    dr.get(url)

    # Scrapping the Web
    soup = BeautifulSoup(dr.page_source, "lxml")


    '''
    Parking Description,Utilities,Dining Room Description,
    Garage,Heating Feature,Cooling Feature,Flooring feature,
    Laundary Feature,Number of stories,Fees,
    Bathroom description,HOA Freq,HOA Name,
    Buyer agency compensation,price/sqft,Redfin Estimate,
    Estimated monthly pay, School ratings, Nearby places count,
    Bike score, Walk Score, Transit score, Level of risk
    factors
    '''
    info = get_house_info(soup)
    property_info.update(info)

    info = extract_school_info(soup)
    property_info.update(info)

    info = get_price_info(soup)
    property_info.update(info)

    info = extract_sales_history(soup)
    property_info.update({'price_history': info})

    info = extract_score(soup,score_type = "Bike Score")
    property_info.update(info)

    info = extract_score(soup,score_type = "Walk Score")
    property_info.update(info)

    info = extract_score(soup,score_type = "Transit Score")
    property_info.update(info)

    info = extract_risk_factors(soup)
    property_info.update(info)

    info = nearby_grocery_park_transit(soup)
    property_info.update(info)

    all_details[url] = property_info
    dr.quit()
    # sleep time has been randomized,so as redfin
    # doesnot identify a pattern and block us
    ts = random.choice([3.5, 2, 4, 2.8, 3.2, 5.3, 6.2, 7.5, 9.3, 1.7, 5.9])
    time.sleep(ts)

processing 0
processing 5
processing 10
processing 15
processing 20
processing 25
processing 30
processing 35
processing 40
processing 45
processing 50
processing 55
processing 60
processing 65
processing 70
processing 75
processing 80
processing 85
processing 90
processing 95


In [ ]:
import copy
#creating a copy
all_details2 = copy.deepcopy(all_details)

In [ ]:
#for making sales price history as part of
#outer dictionary so it will be easy to create
#dataframe at the end
for key_url,val_details in all_details2.items():
    phistory = val_details['price_history']
    #print(phistory)
    val_details.update(phistory)
    del val_details['price_history']

In [ ]:
# making url a part of dictionary values
# would help us when we create dataframe
details = []

for k,v in all_details2.items():
    v.update({'url': k})
    details.append(v)

In [ ]:
details[0:2] # showing two records of the list

[{'PARK_DESCRIPTION': 'Attached Garage',
  'HAS_GARAGE': 2.0,
  'BATH_DESCRIPTION': 'Double Sinks',
  'DINNING_DESC': 'Breakfast Bar, Dining Area in Family Room, Formal Dining Room',
  'FLOORING_FEATURES': 'Hardwood',
  'COOLING_FEATURES': 'Central Forced Air',
  'HEATING_FEATURES': 'Central Forced Air',
  'UTILITIES': 'Public Utilities',
  'NUM_STORIES': 1.0,
  'ELEMENTARY_SCHOOL_RATING': '7',
  'MIDDLE_SCHOOL_RATING': '6',
  'HIGH_SCHOOL_RATING': '6',
  'BUYER_AGENCY_COMPENSATION': 2.0,
  'PRICE_PR_SQFT': 1006.0,
  'REDFIN_ESTIMATE': 2398745.0,
  'EST_MONTHLY_PAY': 17250.0,
  'Bike Score': '37',
  'Bike Score_type': 'Somewhat bikeable',
  'Walk Score': '45',
  'Walk Score_type': 'Car-dependent',
  'Transit Score': '26',
  'Transit Score_type': 'Some transit',
  'Flood Factor': 'Minimal',
  'Fire Factor': 'Moderate',
  'Heat Factor': 'Moderate',
  'Wind Factor': 'Minimal',
  'OVERALL_NEARBY_PLACES': 'Places5 groceries, 21 restaurants, 18 parksTransit27, 64A, 64B, 83',
  'PRICE_HISTORY

In [ ]:
# creating dataframe from the list of dictionary above
df_step2 = pd.DataFrame.from_dict(details)

In [ ]:
df_step2.head()

,PARK_DESCRIPTION,HAS_GARAGE,BATH_DESCRIPTION,DINNING_DESC,FLOORING_FEATURES,COOLING_FEATURES,HEATING_FEATURES,UTILITIES,NUM_STORIES,ELEMENTARY_SCHOOL_RATING,...,PRICE_HISTORY5_PRICE,PRICE_HISTORY6_EVENT_DATE,PRICE_HISTORY6_EVENT_TYPE,PRICE_HISTORY6_PRICE,PRICE_HISTORY7_EVENT_DATE,PRICE_HISTORY7_EVENT_TYPE,PRICE_HISTORY7_PRICE,PRICE_HISTORY8_EVENT_DATE,PRICE_HISTORY8_EVENT_TYPE,PRICE_HISTORY8_PRICE
0,Attached Garage,2.0,Double Sinks,"Breakfast Bar, Dining Area in Family Room, For...",Hardwood,Central Forced Air,Central Forced Air,Public Utilities,1.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Attached Garage, Guest/Visitor Parking, On Street",2.0,"Dual Flush Toilet, Primary (Stall Shower(s)), ...","Dining Area in Living Room, No Formal Dining Room","Carpet, Hardwood, Tile",Central Forced Air,Other (See Remarks),Public Utilities,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Electric Car Hookup,2.0,NaN,Dining Area,NaN,Central Forced Air,Central Forced Air,"Public Utilities, Solar Panels - Leased",1.0,7,...,679500.0,"May 4, 2010",Listed (Active)MLSListings #ML81022077,699900.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Detached,2.0,NaN,Dining Area,Laminate,Central Forced Air,Central Forced Air,Public Utilities,1.0,4,...,NA,"Apr 13, 2022",ContingentMLSListings #ML81884118,NA,"Mar 26, 2022",Listed (Active)MLSListings #ML81884118,1150000.0,NaN,NaN,NaN
4,"Off-Street Parking, Uncovered Parking",NaN,NaN,Other (See Remarks),NaN,Other (See Remarks),Other (See Remarks),Public Utilities,1.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# concatentaing step1 and step2 data to make
#final dataframe
df_final = pd.concat([df_step1, df_step2], axis=1)

In [ ]:
df_final.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,PRICE_HISTORY5_PRICE,PRICE_HISTORY6_EVENT_DATE,PRICE_HISTORY6_EVENT_TYPE,PRICE_HISTORY6_PRICE,PRICE_HISTORY7_EVENT_DATE,PRICE_HISTORY7_EVENT_TYPE,PRICE_HISTORY7_PRICE,PRICE_HISTORY8_EVENT_DATE,PRICE_HISTORY8_EVENT_TYPE,PRICE_HISTORY8_PRICE
0,MLS Listing,NaN,Single Family Residential,6544 Little Falls Dr,San Jose,CA,95120,2378999,4.0,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MLS Listing,NaN,Condo/Co-op,922 Gaspar Vis,San Jose,CA,95126,1200000,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MLS Listing,NaN,Single Family Residential,4993 Grimsby Dr,San Jose,CA,95130,2498888,4.0,2.5,...,679500.0,"May 4, 2010",Listed (Active)MLSListings #ML81022077,699900.0,NaN,NaN,NaN,NaN,NaN,NaN
3,MLS Listing,NaN,Single Family Residential,818 N 13th St,San Jose,CA,95112,1118000,3.0,1.0,...,NA,"Apr 13, 2022",ContingentMLSListings #ML81884118,NA,"Mar 26, 2022",Listed (Active)MLSListings #ML81884118,1150000.0,NaN,NaN,NaN
4,MLS Listing,NaN,Single Family Residential,2451 Cory Ave,San Jose,CA,95128,899999,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Dumping into csv file the final dataframe
df_final.to_csv('redfin_san_jose_final.csv', index=False)

In a similar way, data for each location is extracted in step 2 and then concatenated to the data extracted in step1 to get the final data for each location. Later we would combine final data of all the locations together in the preprocessing stage